# Run this first cell to define all the functions:

In [102]:
# requirements are the folloing:
import math
import fractions
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np

def convertFrac(arr):
    return [fractions.Fraction(i) for i in arr]
def convertBack(arr):
    return [i.numerator/i.denominator for i in arr]

def elemProd(arr1, arr2, sumB=True):
    if not sumB:
        return [i*j for (i, j) in zip(arr1, arr2)]
    return sum([i*j for (i, j) in zip(arr1, arr2)])

def elemSq(arr):
    return [i*i for i in arr]

def convertListToDecimal(listA):
    return [frac if type(frac) == float else (frac if frac.denominator != 1 else frac.numerator) for frac in listA]

def checkForShortSelling(sigma1, sigma2, p12):
    if sigma1 >= sigma2:
        if p12 == 1:
            print("p12 = 1, there exist a deasable set with short selling s.t. sigmaV = 0.  each portfolio has the same sigmaV when sigma1 = sigma2")
        elif (sigma1/sigma2) < p12 < 1:
            print(f"feasable set with short selling, s.t. sigmaV < sigma1 {round(sigma1, 5)}, but without shortselling if sigmaV >= sigma1 {round(sigma1, 5)}")
        elif p12 == sigma1/sigma2:
            print("no short selling")
        elif -1 < p12 < sigma1/sigma2:
            print("feasable set without short selling")
        elif p12 == -1:
            print("there's a feasabale set without short selling, s.t. sigmaV < sigma1")
        else:
            print("wha?")
    else:
        checkForShortSelling(sigma2, sigma1, p12)

def makeCMatrix(correlationList, stdDevList):
    
    matrixSize = len(stdDevList)
    correlationArr = [[0 for _ in range(matrixSize)] for _ in range(matrixSize)]
    for i in range(matrixSize):
        for j in range(matrixSize):
            if i==j:
                correlationArr[i][j] = stdDevList[i]* stdDevList[i]
            else:    
                correlationArr[i][j] = findCorrelation(i+1, j+1, correlationList) * (stdDevList[i] * stdDevList[j])
    
    cMatrix = np.array(correlationArr)
    print("\nC matrix is:")
    print(cMatrix)
    return cMatrix

def findCorrelation(i, j, correlation):
    for val in correlation:
        if val[0] == [i, j] or val[0] == [j, i]:
            return val[1]
                
def meanV(weights, means):
    return sum(w * m for (w, m) in zip(weights, means))

def find_wCwT(w, c):
    wC = np.matmul(w, c)
    print("intermediate calculation: w*C = ", wC)
    return np.matmul(wC, np.array(w).T)
    
def makeM(m, cInv, u):
    m, u = np.array(m), np.array(u)
    mCInv = np.matmul(m, cInv)
    uCInv = np.matmul(u, cInv)
    print("Intermediate Calculations:")
    print("M*C^-1 = ", mCInv)
    print("u*C^-1 = ", uCInv)
    
    mCInv_mT = np.matmul(mCInv, m.T)
    mCInv_uT = np.matmul(mCInv, u.T)
    uCInv_mT = np.matmul(uCInv, m.T)
    uCInv_uT = np.matmul(uCInv, u.T)
    
    M = [[mCInv_mT, uCInv_mT],
        [mCInv_uT, uCInv_uT]]
    
    print("\nhere is M:")
    print(np.array(M))
    return np.array(M)

def minWeights(u, cInv):
    min_weights = np.matmul(u, cInv)/ (np.matmul(np.matmul(u, cInv), np.array(u).T))
    print("\nThe weight in minimum variance portfolio is:")
    print(min_weights)
    return min_weights

def find_W_MVP(m, stdDevs, correlation):
    m, u = np.array(m), np.array([1 for _ in m])
    print("m is:", m, "\nu is:", u)
    cMatrix = np.array(makeCMatrix(correlation, stdDevs))
    cInverse = np.linalg.inv(cMatrix)
    print("\nhere is the inverse of C:\n", cInverse)
    print("\ncalculating M:")
    Mmatrix = makeM(m, cInverse, u)
    print("\nhere is M inverse:")
    MmatrixInv = np.linalg.inv(Mmatrix)
    print(MmatrixInv)
    minimumWeights = minWeights(u, cInverse)
    print("\nThe Expected return is:")
    Emin = elemProd(minimumWeights, m)
    print(Emin)
    print("\nVariance is:")
    wCwT = find_wCwT(minimumWeights, cMatrix)
    print("wCwT = Varinace = ")  
    print(wCwT)
    stdDevMin = math.sqrt(wCwT)
    print("\nThe Std.Dev of the portfolio is:")
    print(stdDevMin)
    
class portfolio_MVP:
    def __init__(this, m, stdDevs, correlation):
        """Given m, stdevs and correlation of the securties in the portfolio, 
        this function prints: C, C^-1, M, M^-1, E[Kv], wCwT = Var(Kv), and stdev(Kv)"""
        this.m = np.array(m)
        this.u = np.array([1 for _ in m])
        this.stdDevs = stdDevs
        this.correlation = correlation
        this.a = -1
        this.calculate_rest()
        
    def calculate_rest(this):
        print("m is:", this.m, "\nu is:", this.u)
        this.CMatrix = np.array(makeCMatrix(this.correlation, this.stdDevs))
        this.CInverse = np.linalg.inv(this.CMatrix)
        print("\nhere is the inverse of C:\n", this.CInverse)
        print("\ncalculating M:")
        this.Mmatrix = makeM(this.m, this.CInverse, this.u)
        print("\nhere is M inverse:")
        this.MmatrixInv = np.linalg.inv(this.Mmatrix)
        print(this.MmatrixInv)
        this.minimumWeights = minWeights(this.u, this.CInverse)
        print("\nThe Expected return of the MVP is:")
        
        this.Emin = elemProd(this.minimumWeights, this.m)
        print(this.Emin)
        print("\nVariance is:")
        this.wCwT = find_wCwT(this.minimumWeights, this.CMatrix)
        print("wCwT = Varinace = ")  
        print(this.wCwT)
        this.stdDevMin = math.sqrt(this.wCwT)
        print("\nThe Std.Dev of the portfolio is:")
        print(this.stdDevMin)
        
    def E_Var_std(this, weights):
        E = elemProd(weights, this.m)
        print(f"expected value is {E}")
        print("\nVariance is:")
        variance = find_wCwT(weights, this.CMatrix)
        print("wCwT = Varinace = ", variance)  
        stdDev = math.sqrt(variance)
        print("\nThe Std.Dev of the portfolio is:")
        print(stdDev)
        return [E, variance, stdDev]
        
    def W_MVP_Given_Ev(this, Ev):
        print("*"*20)
        if this.a == -1:
            this.calculate_ab()
        print("\na, b doesnt change for each portfolio on the minimum variance line")
        print("to get the weights that satisfy a specific E[Kv], \nwe solve the linear equation: w = mu * a + b")
        this.W_given_EKv = this.compute_W_given_E(Ev)
        print("\nVariance is:")
        this.wCwT_given_EKv = find_wCwT(this.W_given_EKv, this.CMatrix)
        print("wCwT = Varinace = ")  
        print(this.wCwT_given_EKv)
        this.stdDevMin_given_EKv = math.sqrt(this.wCwT_given_EKv)
        print("\nThe Std.Dev of the portfolio is:")
        print(this.stdDevMin_given_EKv)
        
        print("*"*20)
        
    def calculate_ab(this):
        print("\ncalculating for a and b:")
        print("[a, b].T = M^-1 * [m, u].T * C^-1")
        this.matrix_ab = np.matmul(np.matmul(this.MmatrixInv, np.array([this.m, this.u])), this.CInverse)
        this.a = this.matrix_ab[0]
        this.b = this.matrix_ab[1]
        print("this is vector a: ", this.a)
        print("this is vector b: ", this.b)
    
    def compute_W_given_E(this, Ev):
        W_given_EKv = (Ev*this.a) + this.b
        print(f"w = {W_given_EKv}")
        return W_given_EKv
        
    def calculate_MVL_slope(this, x, y):
        print("\nfor mu = 0:")
        w0 = this.compute_W_given_E(0)
        print("\nfor mu = 0.1:")
        w01 = this.compute_W_given_E(0.1)
        slope = this.compute_slope(w0, w01, x, y)
        
        # fixed index to be 0 based
        intersect = w0[y-1] - slope*w0[x-1] 
        print(f"\nequation for the line on plane (x var: w{x}, y var: w{y}):")
        print(f" = {slope} * w{x} + {intersect}")
        
        xVal = np.array([0, 0.2, 0.4, 0.6, 0.8, 1])
        yVal = np.array([(slope*xi + intersect) for xi in xVal])
  
        xy_table = pd.DataFrame([xVal, yVal], index=["x axis", "y axis"])
        print("sample points:")
        print(xy_table)
        
    def compute_slope(this, w0, w1, x, y):
        # modifying the indecies
        i, j = x-1, y-1
        print(f"\nmu=0,   w{x} = {w0[i]}")
        print(f"mu=0,   w{y} = {w0[j]}")
        print(f"mu=0.1, w'{x} = {w0[i]}")
        print(f"mu=0.1, w'{y} = {w0[j]}")
        slope = (w1[j] - w0[j]) / (w1[i] - w0[i])
        print(f"\nequation for slope: slope = (w{y} - w'{y}) / (w{x} - w'{x})")
        print(f"slope = {slope}, for graph with (x var: w{x}, y var: w{y})")
        return slope
        
    def market_portfolio(this, R):
        print("*"*20)
        mru = (this.m - R*this.u)
        print("\nweights in a market portfolio = ((m-R*u) * C^-1)/((m-R*u)* C^-1 * uT)")
        print(f"intermediate steps: m-R*u = {mru}")
        print("(m-R*u)* C^-1 = ", np.matmul(mru,this.CInverse))
        this.market_weights = np.matmul(mru,this.CInverse) / np.matmul(np.matmul(mru,this.CInverse), this.u.T)
        print(f"\nmarket portfolio weights = {this.market_weights}")
        this.E_Var_std(this.market_weights)

# 2 securities (K1 and K2), 3 scenarios ==> find everthing else

In [ ]:
p = ["0.2", "0.4", "0.4"]
k1 = ["-0.1", "0","0.2"]
k2 = ["0.05", "0.3", "-0.05"]

p = convertFrac(p)
k1 = convertFrac(k1)
k2 = convertFrac(k2)
E1 = elemProd(p, k1)
E2 = elemProd(p, k2)
E_k1sq = elemProd(p, elemSq(k1))
E_k2sq = elemProd(p, elemSq(k2))
var_k1 = E_k1sq - E1*E1
var_k2 =  E_k2sq - E2*E2
sigmak1 = fractions.Fraction(math.sqrt(var_k1))
sigmak2 = fractions.Fraction(math.sqrt(var_k2))

Ek1k2 = elemProd(elemProd(k1, k2, False), p)
c12 = Ek1k2 - E1*E2
p12 = c12 / (sigmak1*sigmak2)


w1 = (var_k2 - c12) / (var_k1 + var_k2 - 2*c12)
w2 = 1-w1
wfraction = w1/w2
wfraction2 = w2/w1



answers = [E1, E2, E_k1sq, E_k2sq, var_k1, var_k2, sigmak1, sigmak2, Ek1k2, c12, p12, w1, w2, wfraction, wfraction2]
equations = ["E[K1]", "E[K2]", "E[K1^2]", "E[K2^2]", "E(K1^2)- E[K1]^2","E(K2^2)- E[K2]^2", "sqrt(Var(K1))", "sqrt(Var(K2))", 
             "E[K1*K2]", "Cov(K1, K2)", "p12", "w1", "w2", "stdDev(K1)/stdDev(K2)", "stdDev(K2)/stdDev(K1)"]
stringVal = ["E[K1]", "E[K2]", "E[K1^2]", "E[K2^2]", "Var(K1)","Var(K2)", "StdDev(K1)", "StdDev(K2)", 
             "E[K1*K2]", "Cov(K1, K2)", "p12", "w1", "w2", "stdDev(K1)/stdDev(K2)", "stdDev(K2)/stdDev(K1) \n **if std(K1)>std(K2)"]

decimalAnswers = convertBack(answers)

dataLists = [stringVal, equations, decimalAnswers, answers]
df = pd.DataFrame(dataLists, index=["Variable", "Formula", "Value", "fractional Value"]).T
print("")
checkForShortSelling(sigmak1, sigmak2, p12)
df

# HW8 question 1

In [ ]:
# w1, w2, ...., wm
weights = [0.3, 0.1, 0.6]
means = [0.08, 0.1, 0.06]
stdDevs = [0.15, 0.05, 0.12]

# each entry represent correlation of (i, j) ans stored in the form [(i, j), value]
correlation = [
    [[1, 2], 0.3],
    [[2, 3], 0],
    [[1, 3], -0.2],
]


mu_V = meanV(weights, means) 
cMatrix = makeCMatrix(correlation, stdDevs)
wCwT = find_wCwT(weights, cMatrix)
np.set_printoptions(suppress=True)
print("Expected value of portfolio is:", mu_V)
print("Correlation matrix is:")
print(np.array(cMatrix))
print("w*C*wT is: ", wCwT)
print("stdDev of portfolio is", math.sqrt(wCwT))
print("\n", "*"* 20, "\n")
cInverse = np.linalg.inv(cMatrix)
print("here is the inverse of C:")
print(cInverse)
m = np.array(means)
u = np.array([1 for _ in means])
print("m is:", m)
print("u is:", u)
print("calculating M")
Mmatrix = makeM(m, cInverse, u)
print("here is M:")
print(Mmatrix)
print("here is M inverse:")
MmatrixInv = np.linalg.inv(Mmatrix)
print(MmatrixInv)

# HW 7 Part 2:

In [ ]:
m = [0.2, 0.13, 0.17]
stdDevs = [0.25, 0.28, 0.20]
correlation= [
    [[1,2], 0.25],
    [[2,3], 0],
    [[3,1], 0.15]]

find_W_MVP(m, stdDevs, correlation)

# HW 8, Problem 3:

In [ ]:
m = [0.2, 0.1, 0.2]
stdDevs = [0.25, 0.2, 0.25]
correlation= [
    [[1,2], 0.25],
    [[2,3], 0.05],
    [[3,1], 0.15]]

find_W_MVP(m, stdDevs, correlation)

# HW 9 Problem 1

In [ ]:
mu = [0.2, 0.15, 0.17]
stdevs = [0.25, 0.28, 0.2]
corr = [
    [[1, 2],0.25],
    [[2,3], 0.0],
    [[1, 3], 0.15]
    ]
muV = 0.2

x = portfolio_MVP(mu, stdevs, corr)
x.W_MVP_Given_Ev(muV)

# HW 9 Problem 2:

In [ ]:
mu = [0.16, 0.13, 0.13]
stdevs = [0.25, 0.28, 0.2]
corr = [
    [[1, 2],0.3],
    [[2,3], 0.0],
    [[1, 3], 0.15]
    ]

x = portfolio_MVP(mu, stdevs, corr)
x.calculate_ab()
x.calculate_MVL_slope(2, 3)

# HW 9 part 3:

In [ ]:
mu = [0.08, 0.1, 0.09]
stdevs = [0.15, 0.05, 0.12]
corr = [
    [[1, 2],0.3],
    [[2,3], 0.0],
    [[1, 3], -0.2]
    ]
risk_free_rate = 0.05
x = portfolio_MVP(mu, stdevs, corr)
x.market_portfolio(risk_free_rate)
